# 2021.02.14

**-Use AlexNets, but also gonna try with ResNets & VGGNets**   
**-Should shuffle all the data**

<h3><span style = "background-color : red">Color Test</span></h3><br>
    - <strong>Using AlexNet</strong><br>
    - <strong>Using color image data</strong>

In [8]:
import pickle
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

c_F = pickle.load(open("imagedata/seungbin_x_train_F.pickle", "rb"))
c_T = pickle.load(open("imagedata/seungbin_x_train_T.pickle", "rb"))

c_F = np.array(c_F)
c_T = np.array(c_T)

x = np.concatenate( (c_F, c_T, c_F, c_T, c_F, c_T, c_F, c_T, c_F, c_T, c_F, c_T, c_F, c_T, c_F, c_T, c_F, c_T, c_F, c_T), axis = 0 )
# x = x.reshape(-1, 227, 227, 1)

y = [1 for i in range(len(c_F))] + [0 for i in range(len(c_T))] + [1 for i in range(len(c_F))] + [0 for i in range(len(c_T))] + [1 for i in range(len(c_F))] + [0 for i in range(len(c_T))] + [1 for i in range(len(c_F))] + [0 for i in range(len(c_T))] + [1 for i in range(len(c_F))] + [0 for i in range(len(c_T))] + [1 for i in range(len(c_F))] + [0 for i in range(len(c_T))] + [1 for i in range(len(c_F))] + [0 for i in range(len(c_T))] + [1 for i in range(len(c_F))] + [0 for i in range(len(c_T))] + [1 for i in range(len(c_F))] + [0 for i in range(len(c_T))] + [1 for i in range(len(c_F))] + [0 for i in range(len(c_T))]

X = []

for rows in x:
    tmp = cv2.resize(rows, (227, 227))
    X.append(tmp)
    
X = np.array(X)
X = X.reshape(-1, 227, 227, 3)

In [9]:
import warnings
warnings.filterwarnings(action = "ignore")

In [10]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.8)
sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))

In [11]:
INPUT_SHAPE = (227, 227, 3)

model = Sequential()

# 1st Convolutional Layer
model.add( Conv2D(filters=96, input_shape=(INPUT_SHAPE), kernel_size=(11,11), strides=(4,4), padding='valid') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )

# 2nd Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )

# 3rd Convolutional Layer
model.add( Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# 4th Convolutional Layer
model.add( Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# 5th Convolutional Layer
model.add( Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same') )
model.add( Activation('relu') )

# Max Pooling
model.add( MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid') )



########################## Passing it to a Fully Connected layer ##########################
model.add( Flatten() )

K = INPUT_SHAPE[0] * INPUT_SHAPE[1] * INPUT_SHAPE[2]

# 1st Fully Connected Layer
model.add( Dense(4096, input_shape=( K, )) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# 2nd Fully Connected Layer
model.add( Dense(4096) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# 3rd Fully Connected Layer
model.add( Dense(1000) )
model.add( Activation('relu') )
# Add Dropout
model.add( Dropout(0.4) )

# Output Layer
model.add( Dense(2) )
model.add( Activation('softmax') )
          
opt = tf.keras.optimizers.Adam(lr = 1e-3, decay = 1e-5)

model.compile(loss = "sparse_categorical_crossentropy", # or binary_crossentropy
             optimizer = opt,
             metrics = ['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_9 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_10 (Activation)   (None, 27, 27, 256)       0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 13, 13, 384)      

In [12]:
model.fit(X, y, batch_size = 4, epochs = 3, validation_split = 0.1)  # batch_size

Train on 1800 samples, validate on 200 samples
Epoch 1/3
1800/1800 [==============================] - 11s 6ms/sample - loss: 2.3639 - acc: 0.9850 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 2/3
1800/1800 [==============================] - 11s 6ms/sample - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000
Epoch 3/3
1800/1800 [==============================] - 11s 6ms/sample - loss: 0.0000e+00 - acc: 1.0000 - val_loss: 0.0000e+00 - val_acc: 1.0000


<hr><hr>
<strong>Normal CNN same with the last one</strong>
<hr><hr>

In [1]:
import cv2
import warnings
import numpy as np
warnings.filterwarnings(action= "ignore")
import pickle

F1 = pickle.load(open("imagedata/02.seungbin_x_train_F.pickle", "rb"))
T1 = pickle.load(open("imagedata/02.seungbin_x_train_T.pickle", "rb"))

F2 = pickle.load(open("imagedata/03.seungbin_x_train_F.pickle", "rb"))
T2 = pickle.load(open("imagedata/03.seungbin_x_train_T.pickle", "rb"))

F3 = pickle.load(open("imagedata/02.rlawn_x_train_F.pickle", "rb"))
T3 = pickle.load(open("imagedata/02.rlawn_x_train_T.pickle", "rb"))

In [2]:
F1 = np.array(F1)
T1 = np.array(T1)

F2 = np.array(F2)
T2 = np.array(T2)

F3 = np.array(F3)
T3 = np.array(T3)

X = np.concatenate( (F1, T1, F2, T2, F3, T3), axis = 0 )
X = X.reshape(-1, 480, 640, 1)

y = [1 for i in range(len(F1))] + [0 for i in range(len(T1))] + [1 for i in range(len(F2))] + [0 for i in range(len(T2))] + [1 for i in range(len(F3))] + [0 for i in range(len(T3))]

In [3]:
import tensorflow as tf

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.8)
sess = tf.Session(config = tf.ConfigProto(gpu_options = gpu_options))

<hr><hr>

## If the error called "OOM (ResourceExhausted)" occurs,
+ or **'Root 2 blah-blah error' occurs**,
### 1) try to cut down the batch size, or
### 2) increase the value (percentage) of "per_process_gpu_fraction" function.   
https://stackoverflow.com/questions/59394947/how-to-fix-resourceexhaustederror-oom-when-allocating-tensor/59395251

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import pickle

# X = pickle.load(open("X.pickle", "rb"))
# y = pickle.load(open("Y.pickle", "rb"))

X = X/255.0

model = Sequential()
X.shape[1:]

(480, 640, 1)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
import pickle

# X = pickle.load(open("X.pickle", "rb"))
# y = pickle.load(open("Y.pickle", "rb"))

X = X/255.0

model = Sequential()

## Layer 1
model.add(   Conv2D(64, (3,3), input_shape = X.shape[1:])   )  # 64 units, 3x3 windowsize (for next)
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

## Layer 2
model.add(   Conv2D(64, (3,3))   )
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size = (2, 2)))

## Layer 3
## 2D from CNN into 1D for Dense layer
model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))

model.add(Dense(1))  # output layer
model.add(Activation("sigmoid"))

model.compile(loss = "binary_crossentropy",
             optimizer = "adam",
             metrics = ['accuracy'])

model.fit(X, y, batch_size = 4, epochs = 3, validation_split = 0.1)  # batch_size

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 13500 samples, validate on 1500 samples
Epoch 1/3
13500/13500 [==============================] - 437s 32ms/sample - loss: 0.0960 - acc: 0.9841 - val_loss: 0.2189 - val_acc: 0.9247
Epoch 2/3
13500/13500 [==============================] - 445s 33ms/sample - loss: 0.0268 - acc: 0.9970 - val_loss: 0.4073 - val_acc: 0.9107
Epoch 3/3
13500/13500 [==============================] - 445s 33ms/sample - loss: 0.0233 - acc: 0.9953 - val_loss: 0.7591 - val_acc: 0.8493


In [23]:
# model.save("0213.model")

## Load the Test Set

In [5]:
test_F1 = pickle.load(open("imagedata/02.seungbin_x_test_F.pickle", "rb"))
test_T1 = pickle.load(open("imagedata/02.seungbin_x_test_T.pickle", "rb"))

test_F2 = pickle.load(open("imagedata/03.seungbin_x_test_F.pickle", "rb"))
test_T2 = pickle.load(open("imagedata/03.seungbin_x_test_T.pickle", "rb"))

test_F3 = pickle.load(open("imagedata/02.rlawn_x_test_F.pickle", "rb"))
test_T3 = pickle.load(open("imagedata/02.rlawn_x_test_T.pickle", "rb"))

test_F1 = np.array(test_F1)
test_T1 = np.array(test_T1)

test_F2 = np.array(test_F2)
test_T2 = np.array(test_T2)

test_F3 = np.array(test_F3)
test_T3 = np.array(test_T3)

tmp_test = np.concatenate( (test_F1, test_T1, test_F2, test_T2, test_F3, test_T3), axis = 0 )
tmp_test = tmp_test.reshape(-1, 480, 640, 1)

In [6]:
tmp_test.shape

(3500, 480, 640, 1)

## Print the prediction on test set 'one by one' considering memory short

In [7]:
tmp_test[0].shape

(480, 640, 1)

In [8]:
print(tmp_test[0].shape)

(480, 640, 1)


In [9]:
pose = ["true", "false"]

for interval in range(0, 3500, 1):
    predictions = model.predict(tmp_test[interval:interval+1])
    print(pose[int(predictions[0])])

false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
true
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false

false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
false
fals

true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
false
true
true
true
true
true
true
true
true
true
false
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
false
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
false
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
true
false
true
true
true
true
true
false
true
true
true
true
true
true
false
true
true
true
true
true
true
true
true
true
true
tru